# Упражнение 04 : A / B-тестирование

In [161]:
import pandas as pd
import sqlite3

## создайте подключение к базе данных с помощью библиотеки sqlite3

In [162]:
con = sqlite3.connect('../data/checking-logs.sqlite')

## используя только один запрос для каждой из групп, создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками
* время должно иметь значения: после и до
* avg_diff содержит среднюю дельту для всех пользователей за период до того, как каждый из них впервые посетил страницу, и после
* учитывайте только тех пользователей, у которых есть наблюдения до и после
## мы по-прежнему не используем лабораторию ’project1’

In [163]:
sql = '''
SELECT time, AVG(avg_diff) AS avg_diff FROM
(SELECT
    uid,
    CASE
        WHEN first_commit_ts < first_view_ts THEN 'before'
        WHEN first_commit_ts > first_view_ts THEN 'after'
    END as time,
    AVG((deadlines.deadlines - strftime('%s', first_commit_ts)) / 3600) AS avg_diff
FROM test
JOIN deadlines
ON test.labname = deadlines.labs
WHERE
    test.labname != 'project1'
GROUP BY uid)
GROUP BY time
'''
test_results = pd.read_sql(sql, con)
test_results

,time,avg_diff
0,after,110.162500
1,before,78.388095


In [164]:
sql = '''
SELECT time, AVG(avg_diff) AS avg_diff FROM
(SELECT
    uid,
    CASE
        WHEN first_commit_ts < first_view_ts THEN 'before'
        WHEN first_commit_ts > first_view_ts THEN 'after'
    END as time,
    AVG((deadlines.deadlines - strftime('%s', first_commit_ts)) / 3600) AS avg_diff
FROM control
JOIN deadlines
ON control.labname = deadlines.labs
WHERE
    control.labname != 'project1'
GROUP BY uid)
GROUP BY time
'''
control_results = pd.read_sql(sql, con)
control_results

,time,avg_diff
0,after,67.166667
1,before,96.130000


## закройте соединение

In [167]:
con.close()

## У вас есть ответ: оказалась ли гипотеза верной и влияет ли страница на поведение учащихся?